In [1]:
# ==========================================
# Global model (LGBM) + Baselines
# Target week: 2025-12-29  ✅ (train only uses data before this week)
# Save outputs to ../../result/random/
# ==========================================

import os
import numpy as np
import pandas as pd
import lightgbm as lgb

# --------------------------
# Config
# --------------------------
DATA_PATH = "../../final_data/data_260119_random.csv"
TARGET_WEEK = "2025-12-29"   # ✅ changed

CAT_COLS = ["treatment", "source", "ops_type_merged", "city_group"]
DATE_COL = "experiment_date"

Y1 = "trip_cnt_per_user"
Y2 = "nonrepeat_cnt_per_user"

Y1_LAG1 = "trip_cnt_per_user_lag1"
Y1_ROLL4 = "trip_cnt_per_user_roll4"
Y2_LAG1 = "nonrepeat_cnt_per_user_lag1"
Y2_ROLL4 = "nonrepeat_cnt_per_user_roll4"

DROP_FEATURES = {"log1p_user_cnt", "week_idx"}
RANDOM_STATE = 42

OUTPUT_DIR = os.path.join("..", "..", "result", "1229")
ENCODING_EXCEL = "utf-8-sig"

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

# --------------------------
# Helpers
# --------------------------
def rmse(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

def mae(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.mean(np.abs(y_true - y_pred)))

def make_time_val_split(train_df, date_col, n_val_weeks=4):
    uniq_dates = np.array(sorted(pd.to_datetime(train_df[date_col]).dt.date.unique()))
    if len(uniq_dates) <= 2:
        return train_df.copy(), None

    n_val_weeks = min(n_val_weeks, max(1, len(uniq_dates) // 5))
    val_dates = set(uniq_dates[-n_val_weeks:])

    tr = train_df[~pd.to_datetime(train_df[date_col]).dt.date.isin(val_dates)].copy()
    va = train_df[pd.to_datetime(train_df[date_col]).dt.date.isin(val_dates)].copy()

    if tr.empty or va.empty:
        cut = int(len(uniq_dates) * 0.8)
        cut = max(1, min(cut, len(uniq_dates) - 1))
        tr_dates = set(uniq_dates[:cut])
        va_dates = set(uniq_dates[cut:])
        tr = train_df[pd.to_datetime(train_df[date_col]).dt.date.isin(tr_dates)].copy()
        va = train_df[pd.to_datetime(train_df[date_col]).dt.date.isin(va_dates)].copy()
        if tr.empty or va.empty:
            return train_df.copy(), None

    return tr, va

def safe_mkdir(path):
    os.makedirs(path, exist_ok=True)

def save_csv(df_, filename):
    path = os.path.join(OUTPUT_DIR, filename)
    df_.to_csv(path, index=False, encoding=ENCODING_EXCEL)
    return path

def metrics_by_group(test_df, group_col, y_col, pred_cols):
    rows = []
    for key, g in test_df.groupby(group_col):
        y = g[y_col].astype(float).values
        n = len(g)
        for model_name, pred_col in pred_cols.items():
            rows.append({
                group_col: key,
                "target": y_col,
                "model": model_name,
                "MAE": mae(y, g[pred_col].astype(float).values),
                "RMSE": rmse(y, g[pred_col].astype(float).values),
                "n": n,
            })
    return pd.DataFrame(rows).sort_values(["target", group_col, "model"]).reset_index(drop=True)

def group_rmse_table(test_df, y_col, pred_col, group_cols):
    rows = []
    for key, g in test_df.groupby(group_cols):
        y = g[y_col].astype(float).values
        p = g[pred_col].astype(float).values
        rows.append({
            **{col: val for col, val in zip(group_cols, key)},
            "target": y_col,
            "RMSE": rmse(y, p),
            "MAE": mae(y, p),
            "n": len(g),
        })
    return pd.DataFrame(rows).sort_values(["target", "RMSE"]).reset_index(drop=True)

def print_and_save(df_, title, filename, head_n=None):
    print(f"\n=== {title} ===")
    if head_n is None:
        print(df_.to_string(index=False))
    else:
        print(df_.head(head_n).to_string(index=False))
        if len(df_) > head_n:
            print(f"... (printed first {head_n} rows; total rows={len(df_)})")
    path = save_csv(df_, filename)
    print(f"Saved: {os.path.abspath(path)}")
    return path

def enforce_train_categories(train_df, test_df, cat_cols, unk="__UNK__"):
    """
    ✅ Avoid 'peeking' future categories:
    - categories are defined ONLY from train
    - test unseen categories -> __UNK__
    """
    train_df = train_df.copy()
    test_df = test_df.copy()

    for c in cat_cols:
        # use string to avoid category union leakage
        tr = train_df[c].astype("string")
        te = test_df[c].astype("string")

        tr_levels = pd.Index(tr.dropna().unique())
        if unk not in tr_levels:
            tr_levels = tr_levels.append(pd.Index([unk]))

        # map unseen in test to unk
        te = te.where(te.isin(tr_levels), other=unk)

        train_df[c] = pd.Categorical(tr, categories=tr_levels)
        test_df[c]  = pd.Categorical(te, categories=tr_levels)

    return train_df, test_df

# --------------------------
# Load & preprocess
# --------------------------
df = pd.read_csv(DATA_PATH)
df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")

if df[DATE_COL].isna().any():
    bad = df[df[DATE_COL].isna()].head(10)
    raise ValueError(f"Found unparsable dates in {DATE_COL}. Examples:\n{bad}")

required_cols = CAT_COLS + [DATE_COL, Y1, Y2, Y1_LAG1, Y1_ROLL4, Y2_LAG1, Y2_ROLL4]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

min_date = df[DATE_COL].min()
df["week_idx"] = ((df[DATE_COL] - min_date).dt.days // 7).astype(int)

target_week = pd.to_datetime(TARGET_WEEK)

train = df[df[DATE_COL] < target_week].copy()  # ✅ strictly before
test = df[df[DATE_COL] == target_week].copy()

if train.empty:
    raise ValueError("Train set is empty. TARGET_WEEK may be too early.")
if test.empty:
    raise ValueError(f"Test set is empty for week {TARGET_WEEK}. Check experiment_date values.")

# ✅ avoid future category leakage
train, test = enforce_train_categories(train, test, CAT_COLS, unk="__UNK__")

print("=== Target week ===", TARGET_WEEK)
print("Train rows:", len(train), "| Test rows:", len(test))

# --------------------------
# Baselines
# --------------------------
test["pred_y1_lag1"] = test[Y1_LAG1].astype(float)
test["pred_y1_roll4"] = test[Y1_ROLL4].astype(float)
test["pred_y2_lag1"] = test[Y2_LAG1].astype(float)
test["pred_y2_roll4"] = test[Y2_ROLL4].astype(float)

# --------------------------
# Features
# --------------------------
exclude = {Y1, Y2} | set(DROP_FEATURES)

# ✅ safer: feature cols derived from TRAIN columns only
feature_cols = [c for c in train.columns if c not in exclude and c != DATE_COL]

cat_features = [c for c in CAT_COLS if c in feature_cols]

X_train = train[feature_cols].copy()
X_test = test[feature_cols].copy()

y1_train = train[Y1].astype(float).values
y2_train = train[Y2].astype(float).values
y1_true = test[Y1].astype(float).values
y2_true = test[Y2].astype(float).values

# --------------------------
# Val split for early stopping
# --------------------------
tr_df, va_df = make_time_val_split(train, DATE_COL, n_val_weeks=4)
X_tr = tr_df[feature_cols].copy()
y1_tr = tr_df[Y1].astype(float).values
y2_tr = tr_df[Y2].astype(float).values

X_va = y1_va = y2_va = None
if va_df is not None:
    X_va = va_df[feature_cols].copy()
    y1_va = va_df[Y1].astype(float).values
    y2_va = va_df[Y2].astype(float).values

fit_kwargs = dict(categorical_feature=cat_features)

# --------------------------
# Same params as your original
# --------------------------
base_params = dict(
    objective="regression",
    n_estimators=8000,
    learning_rate=0.03,
    random_state=RANDOM_STATE,
    force_row_wise=True,
)

params_y1 = dict(
    num_leaves=31, max_depth=6, min_child_samples=300,
    subsample=0.8, colsample_bytree=0.8, bagging_freq=1,
    reg_alpha=0.0, reg_lambda=2.0, min_split_gain=0.0,
)

params_y2 = dict(
    num_leaves=31, max_depth=6, min_child_samples=200,
    subsample=0.8, colsample_bytree=0.9, bagging_freq=1,
    reg_alpha=0.0, reg_lambda=2.0, min_split_gain=0.0,
)

m_y1 = lgb.LGBMRegressor(**{**base_params, **params_y1})
m_y2 = lgb.LGBMRegressor(**{**base_params, **params_y2})

if X_va is not None:
    m_y1.fit(
        X_tr, y1_tr,
        eval_set=[(X_va, y1_va)],
        eval_metric="l2",
        callbacks=[lgb.early_stopping(stopping_rounds=200, verbose=False)],
        **fit_kwargs
    )
    m_y2.fit(
        X_tr, y2_tr,
        eval_set=[(X_va, y2_va)],
        eval_metric="l2",
        callbacks=[lgb.early_stopping(stopping_rounds=200, verbose=False)],
        **fit_kwargs
    )
else:
    m_y1.fit(X_train, y1_train, **fit_kwargs)
    m_y2.fit(X_train, y2_train, **fit_kwargs)

test["pred_y1_lgbm"] = m_y1.predict(X_test)
test["pred_y2_lgbm"] = m_y2.predict(X_test)

# --------------------------
# Overall metrics
# --------------------------
metrics_df = pd.DataFrame([
    {"target": Y1, "model": "baseline_lag1",  "MAE": mae(y1_true, test["pred_y1_lag1"]),  "RMSE": rmse(y1_true, test["pred_y1_lag1"])},
    {"target": Y1, "model": "baseline_roll4", "MAE": mae(y1_true, test["pred_y1_roll4"]), "RMSE": rmse(y1_true, test["pred_y1_roll4"])},
    {"target": Y1, "model": "lgbm_global",    "MAE": mae(y1_true, test["pred_y1_lgbm"]),  "RMSE": rmse(y1_true, test["pred_y1_lgbm"])},
    {"target": Y2, "model": "baseline_lag1",  "MAE": mae(y2_true, test["pred_y2_lag1"]),  "RMSE": rmse(y2_true, test["pred_y2_lag1"])},
    {"target": Y2, "model": "baseline_roll4", "MAE": mae(y2_true, test["pred_y2_roll4"]), "RMSE": rmse(y2_true, test["pred_y2_roll4"])},
    {"target": Y2, "model": "lgbm_global",    "MAE": mae(y2_true, test["pred_y2_lgbm"]),  "RMSE": rmse(y2_true, test["pred_y2_lgbm"])},
]).sort_values(["target", "model"]).reset_index(drop=True)

# --------------------------
# Metrics by ops_type_merged / city_group / treatment
# --------------------------
pred_map_y1 = {"baseline_lag1": "pred_y1_lag1", "baseline_roll4": "pred_y1_roll4", "lgbm_global": "pred_y1_lgbm"}
pred_map_y2 = {"baseline_lag1": "pred_y2_lag1", "baseline_roll4": "pred_y2_roll4", "lgbm_global": "pred_y2_lgbm"}

metrics_by_ops_df = pd.concat([
    metrics_by_group(test, "ops_type_merged", Y1, pred_map_y1),
    metrics_by_group(test, "ops_type_merged", Y2, pred_map_y2),
], ignore_index=True).sort_values(["target", "ops_type_merged", "model"]).reset_index(drop=True)

metrics_by_city_df = pd.concat([
    metrics_by_group(test, "city_group", Y1, pred_map_y1),
    metrics_by_group(test, "city_group", Y2, pred_map_y2),
], ignore_index=True).sort_values(["target", "city_group", "model"]).reset_index(drop=True)

metrics_by_treat_df = pd.concat([
    metrics_by_group(test, "treatment", Y1, pred_map_y1),
    metrics_by_group(test, "treatment", Y2, pred_map_y2),
], ignore_index=True).sort_values(["target", "treatment", "model"]).reset_index(drop=True)

# --------------------------
# FULL prediction output
# --------------------------
id_cols = [DATE_COL, "treatment", "source", "ops_type_merged", "city_group"]
pred_cols = [
    Y1, "pred_y1_lag1", "pred_y1_roll4", "pred_y1_lgbm",
    Y2, "pred_y2_lag1", "pred_y2_roll4", "pred_y2_lgbm",
]
full_pred_df = test[id_cols + pred_cols].copy()

# --------------------------
# Aggregates
# --------------------------
agg = pd.DataFrame({
    "metric": ["mean_true", "mean_pred_lag1", "mean_pred_roll4", "mean_pred_lgbm"],
    "trip_cnt_per_user": [
        float(np.mean(test[Y1])),
        float(np.mean(test["pred_y1_lag1"])),
        float(np.mean(test["pred_y1_roll4"])),
        float(np.mean(test["pred_y1_lgbm"])),
    ],
    "nonrepeat_cnt_per_user": [
        float(np.mean(test[Y2])),
        float(np.mean(test["pred_y2_lag1"])),
        float(np.mean(test["pred_y2_roll4"])),
        float(np.mean(test["pred_y2_lgbm"])),
    ],
})

# --------------------------
# Best/Worst 15 group RMSE (LGBM only)
# --------------------------
GROUP_KEY_COLS = ["treatment", "source", "ops_type_merged", "city_group"]

group_rmse_y1 = group_rmse_table(test, Y1, "pred_y1_lgbm", GROUP_KEY_COLS)
group_rmse_y2 = group_rmse_table(test, Y2, "pred_y2_lgbm", GROUP_KEY_COLS)

best15_y1 = group_rmse_y1.nsmallest(15, "RMSE").copy()
worst15_y1 = group_rmse_y1.nlargest(15, "RMSE").copy()
best15_y2 = group_rmse_y2.nsmallest(15, "RMSE").copy()
worst15_y2 = group_rmse_y2.nlargest(15, "RMSE").copy()

# --------------------------
# Save outputs (add week tag to filenames to avoid overwriting)
# --------------------------
safe_mkdir(OUTPUT_DIR)
TAG = TARGET_WEEK.replace("-", "")

print_and_save(metrics_df, "Overall metrics (target week)", f"random_{TAG}_overall_metrics.csv", head_n=None)

print_and_save(metrics_by_ops_df, "Metrics by ops_type_merged (target week)", f"random_{TAG}_metrics_by_ops_type_merged.csv", head_n=80)
print_and_save(metrics_by_city_df, "Metrics by city_group (target week)", f"random_{TAG}_metrics_by_city_group.csv", head_n=80)
print_and_save(metrics_by_treat_df, "Metrics by treatment (target week)", f"random_{TAG}_metrics_by_treatment.csv", head_n=80)

print_and_save(full_pred_df, "Predictions FULL (saved full; preview shown above)", f"random_{TAG}_predictions_full.csv", head_n=30)

print_and_save(agg, "Quick aggregates (means)", f"random_{TAG}_quick_aggregates_means.csv", head_n=None)

print_and_save(group_rmse_y1, "Group RMSE (LGBM) - trip_cnt_per_user (all groups)", f"random_{TAG}_group_rmse_lgbm_trip_all_groups.csv", head_n=30)
print_and_save(group_rmse_y2, "Group RMSE (LGBM) - nonrepeat_cnt_per_user (all groups)", f"random_{TAG}_group_rmse_lgbm_nonrepeat_all_groups.csv", head_n=30)

print_and_save(best15_y1, "Best 15 group RMSE (LGBM) - trip_cnt_per_user", f"random_{TAG}_best15_group_rmse_lgbm_trip.csv", head_n=None)
print_and_save(worst15_y1, "Worst 15 group RMSE (LGBM) - trip_cnt_per_user", f"random_{TAG}_worst15_group_rmse_lgbm_trip.csv", head_n=None)
print_and_save(best15_y2, "Best 15 group RMSE (LGBM) - nonrepeat_cnt_per_user", f"random_{TAG}_best15_group_rmse_lgbm_nonrepeat.csv", head_n=None)
print_and_save(worst15_y2, "Worst 15 group RMSE (LGBM) - nonrepeat_cnt_per_user", f"random_{TAG}_worst15_group_rmse_lgbm_nonrepeat.csv", head_n=None)

params_log = pd.DataFrame([
    {"target": Y1, "params": str({**base_params, **params_y1})},
    {"target": Y2, "params": str({**base_params, **params_y2})},
])
print_and_save(params_log, "Final model params", f"random_{TAG}_final_model_params.csv", head_n=None)

print(f"\n✅ Saved all outputs to: {os.path.abspath(OUTPUT_DIR)}")


=== Target week === 2025-12-29
Train rows: 1920 | Test rows: 192
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Info] Total Bins 11852
[LightGBM] [Info] Number of data points in the train set: 1536, number of used features: 73
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Info] Start training from score 0.108608
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

C:\Users\minhsiang.chang\AppData\Local\Temp\ipykernel_21944\4236329392.py:85: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for key, g in test_df.groupby(group_col):
c:\Users\minhsiang.chang\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\minhsiang.chang\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\_core\_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\minhsiang.chang\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\minhsiang.chang\A

Saved: d:\minhsiang.chang\Desktop\2026winter_project\result\1229\random_20251229_metrics_by_city_group.csv

=== Metrics by treatment (target week) ===
treatment                 target          model      MAE     RMSE  n
  15x2元1張 nonrepeat_cnt_per_user  baseline_lag1 0.035470 0.049439 24
  15x2元1張 nonrepeat_cnt_per_user baseline_roll4 0.033861 0.047408 24
  15x2元1張 nonrepeat_cnt_per_user    lgbm_global 0.031989 0.045341 24
    15元1張 nonrepeat_cnt_per_user  baseline_lag1 0.049072 0.105623 24
    15元1張 nonrepeat_cnt_per_user baseline_roll4 0.039636 0.069879 24
    15元1張 nonrepeat_cnt_per_user    lgbm_global 0.028072 0.038606 24
  20x2元1張 nonrepeat_cnt_per_user  baseline_lag1 0.044220 0.072362 24
  20x2元1張 nonrepeat_cnt_per_user baseline_roll4 0.047286 0.076599 24
  20x2元1張 nonrepeat_cnt_per_user    lgbm_global 0.040469 0.065028 24
    20元1張 nonrepeat_cnt_per_user  baseline_lag1 0.078896 0.112992 24
    20元1張 nonrepeat_cnt_per_user baseline_roll4 0.058355 0.089472 24
    20元1張 nonrepeat_c